In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from scipy.stats import describe
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from numpy.random import *
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.pipeline import FeatureUnion
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import ShuffleSplit
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from pandas import DataFrame
from pandas import concat
import pickle
from pandas import DataFrame
from pandas import concat

def series_to_supervised(data,columns,index, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [(columns[j]+'(t-%d)' % (i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [(columns[j]+'(t)')for j in range(n_vars)]
		else:
			names += [(columns[j]+'(t+%d)' % (i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	agg.index = index
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg


def read_data(obj):
    ## get ticker data
    ticker = pd.read_csv('data/'+obj+'_ticker.csv', index_col=0, header=None)
    ticker.index = pd.to_datetime(ticker.index, unit='s')
    ticker.index.name = None
    ticker.columns = [
        'sell_highest_price', 
        'sell_highest_vol', 
        'buy_highest_price', 
        'buy_highest_vol', 
        'last_trade_price', 
        'daily_trade_vol', 
        'daily_high_price', 
        'daily_low_price'
    ]
    ticker.values[np.where(ticker.values <= 0)] = np.nan
    ticker = ticker.interpolate()
    ## trades data
    trades = pd.read_csv('data/'+obj+'_trades.csv', index_col=0, header=None) 

    ## get book data
    book = pd.read_csv('data/'+obj+'_book.csv', index_col=0, header=None)    
    ticker = ticker.add_suffix(obj)
    return ticker,trades,book


In [3]:
metals = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']
ticker = {'A':[], 'B':[], 'C':[], 'D':[], 'E':[], 'F':[], 'G':[], 'H':[], 'I':[]}
trades = {'A':[], 'B':[], 'C':[], 'D':[], 'E':[], 'F':[], 'G':[], 'H':[], 'I':[]}
book = {'A':[], 'B':[], 'C':[], 'D':[], 'E':[], 'F':[], 'G':[], 'H':[], 'I':[]}
models = {'A':[], 'B':[], 'C':[], 'D':[], 'E':[], 'F':[], 'G':[], 'H':[], 'I':[]}
for currency in metals:
    ticker[currency],trades[currency],book[currency] = read_data(currency)

In [6]:
for cur_met in metals:
    ########## get statistics of each 5 min in vopr - price and vol trade 
    trades[cur_met].replace(-1,np.NaN)

    temp_price_trade = trades[cur_met].iloc[:,3::4]
    temp_vol_trade = trades[cur_met].iloc[:,2::4]
    temp_vopr_trade = pd.DataFrame(trades[cur_met].iloc[:,3::4].values * trades[cur_met].iloc[:,2::4].values)
    temp = np.sum(np.absolute(temp_vopr_trade.get_values()),axis=1) / np.sum(np.absolute(temp_vol_trade.get_values()),axis=1)
    absvopr = {'absvopr' : temp}
    absvopr = pd.DataFrame.from_dict(absvopr)

    K = 10
    ### set index of dataframe to time and merger data 
    absvopr.index = ticker[cur_met].index
    ### merge step ticker and trade
    imp_ticker = ticker[cur_met].iloc[:,[0,2,4,7]]    
    stock = pd.DataFrame(index=ticker[cur_met].index)
    stock = pd.merge(imp_ticker,absvopr, left_index=True, right_index=True) ## y
    other_stock = pd.DataFrame(index=ticker[cur_met].index)
    for currency in metals:
        if(currency!=cur_met):
            ticker[currency].index = ticker[cur_met].index
            temp_tic = ticker[currency].iloc[:,:]
            other_stock = pd.merge(other_stock,temp_tic, left_index=True, right_index=True)
    other_stock.head()

    all_feat = [u'sell_highest_price',u'buy_highest_price',u'last_trade_price']
    all_feat = [x+cur_met for x in all_feat]
    all_feat.append(u'absvopr')     

    temp_stock = stock.loc[:,all_feat];
    data_stock_raw = series_to_supervised(temp_stock.values,temp_stock.columns.get_values(),temp_stock.index,25, 11)
    other_data_raw = other_stock
   
    #### set feature name 
    input_feat = temp_stock.columns
    past_feat = [0,1,2,10]
    input_names = []
    for i in past_feat:
        if (i==0):
            input_names += [(input_feat[j]+'(t)')for j in range(len(input_feat))]
        else:
             input_names += [(input_feat[j]+'(t-%d)' % (i))for j in range(len(input_feat))]


    output_feat = temp_stock.columns[2]
    fut_feat = list(range(10))
    fut_feat = []
    for i in range(1,11):
         temp = []
         temp.append(output_feat+'(t+%d)' % (i))
         fut_feat.append(temp)

    f_model1 =ExtraTreesRegressor(n_estimators=70, max_features='sqrt',n_jobs=-1,
                                        random_state=0)
    f_model2 =ExtraTreesRegressor(n_estimators=100, max_features='sqrt',n_jobs=-1,
                                        random_state=0)
    # create pipeline
    estimators = []
    estimators.append(('feature_selection', SelectFromModel(f_model1)))
    estimators.append(('lin', f_model2))
    pip = Pipeline(estimators)
    
    data =  pd.concat([other_data_raw, data_stock_raw.loc[:,input_names]], axis=1, join='inner')

    ##### produce input
    poly = PolynomialFeatures(interaction_only=True,include_bias = False)
    X_t = data.loc[:,input_names].get_values()
    X_t = StandardScaler().fit_transform(X_t)    
    X_t = poly.fit_transform(X_t)
    Y = data_stock_raw.loc[:,sum(fut_feat, [])].get_values()
    pip.fit(X_t,Y)
    models[cur_met] = pip
    print(cur_met)

A
B
C
D
E
F
G
H
I


In [7]:
pkl_filename = "my_model.pkl"  
with open(pkl_filename, 'wb') as file:  
    pickle.dump(models, file)

In [12]:
r = ticker['A']
r

,sell_highest_priceA,sell_highest_volA,buy_highest_priceA,buy_highest_volA,last_trade_priceA,daily_trade_volA,daily_high_priceA,daily_low_priceA
2017-11-13 06:47:52.005880000,70.539792,17.447675,70.555797,85.097402,70.549395,40782.079221,75.962564,68.647903
2017-11-13 06:52:57.527236999,70.338118,32.369336,70.346655,49.160407,70.346655,40795.426010,75.962564,68.647903
2017-11-13 06:57:58.085662000,70.367996,39.523229,70.396806,24.890509,70.367996,40807.995117,75.962564,68.647903
2017-11-13 07:02:57.098389000,70.339185,21.258633,70.340252,26.787416,70.340252,40787.892639,75.962564,68.647903
2017-11-13 07:07:57.429270000,70.199401,69.394319,70.207938,39.864839,70.207938,40847.685300,75.962564,68.647903
2017-11-13 07:12:57.096186000,70.160987,73.978804,70.162054,38.047741,70.159920,40993.393362,75.962564,68.647903
2017-11-13 07:17:57.096133000,70.004130,68.230823,70.005197,39.548353,70.005197,41106.701654,75.962564,68.647903
2017-11-13 07:22:57.322941000,70.410678,74.828267,70.411745,38.636847,70.410678,41131.607149,75.962564,68.647903
2017-11-13 07:27:57.149332000,70.270894,23.475966,70.271961,48.984871,70.239949,41181.716558,75.962564,68.647903
2017-11-13 07:32:57.085762000,70.302905,30.283577,70.303972,32.589320,70.303972,41076.707075,75.962564,68.647903
